In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! uv pip uninstall numpy scipy gensim
! uv pip install numpy==1.26.4

Using Python 3.11.12 environment at: /usr
Uninstalled 3 packages in 291ms
 - gensim==4.3.3
 - numpy==1.26.4
 - scipy==1.13.1
Using Python 3.11.12 environment at: /usr
Resolved 1 package in 5ms
Installed 1 package in 29ms
 + numpy==1.26.4


In [ ]:
# Load preprocessed features
DPPpath = '/content/drive/My Drive/Colab/AS4/STEP2-feature_Engineering/'
# Load preprocessed features
df_tfidf = pd.read_pickle(DPPpath + 'features_tfidf.pkl')
df_bow = pd.read_pickle(DPPpath + 'features_bow.pkl')
df_ngram = pd.read_pickle(DPPpath + 'features_ngram2.pkl')
df_lda = pd.read_pickle(DPPpath + 'features_lda.pkl')

# Prepare data
labels = df_tfidf['label']
numeric_labels = df_tfidf['label_num'].values
classes = np.unique(numeric_labels)
from sklearn.preprocessing import label_binarize
y_binarized = label_binarize(numeric_labels, classes=classes)

# Feature matrices (drop label columns)
X_tfidf = df_tfidf.drop(['label', 'label_num'], axis=1).values
X_bow = df_bow.drop(['label', 'label_num'], axis=1).values
X_ngram = df_ngram.drop(['label', 'label_num'], axis=1).values
X_lda = df_lda.drop(['label', 'label_num'], axis=1).values

# Combine features function
def combine_features(*arrays):
    return np.hstack(arrays)

# Dictionary of feature sets (include individual and combinations)
feature_sets = {
    'TF-IDF': X_tfidf,
    'BoW': X_bow,
    'N-grams': X_ngram,
    'LDA': X_lda,
    'TF-IDF + BoW': combine_features(X_tfidf, X_bow),
    'TF-IDF + N-grams': combine_features(X_tfidf, X_ngram),
    'TF-IDF + LDA': combine_features(X_tfidf, X_lda),
    'BoW + N-grams': combine_features(X_bow, X_ngram),
    'BoW + LDA': combine_features(X_bow, X_lda),
    'N-grams + LDA': combine_features(X_ngram, X_lda),
    'TF-IDF + BoW + N-grams': combine_features(X_tfidf, X_bow, X_ngram),
    'TF-IDF + BoW + LDA': combine_features(X_tfidf, X_bow, X_lda),
    'TF-IDF + N-grams + LDA': combine_features(X_tfidf, X_ngram, X_lda),
    'BoW + N-grams + LDA': combine_features(X_bow, X_ngram, X_lda),
    'All Combined': combine_features(X_tfidf, X_bow, X_ngram, X_lda),
}
y = labels

NameError: name 'pd' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
import numpy as np

# Example inputs:
# feature_sets = {'BoW': X_bow, 'TF-IDF': X_tfidf, 'LDA': X_lda}
# numeric_labels = integer encoded labels (via LabelEncoder)
# y_binarized = label_binarize(numeric_labels, classes=...)

kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
results = {}

for name, X in feature_sets.items():
    accuracies = []
    roc_aucs = []

    print(f"\nTraining and evaluating with feature set: {name}")
    for fold, (train_index, test_index) in enumerate(kf.split(X, numeric_labels), 1):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = numeric_labels[train_index], numeric_labels[test_index]
        y_test_binarized = y_binarized[test_index]

        # Initialize Random Forest
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)

        acc = accuracy_score(y_test, y_pred)

        # Handle binary/multiclass ROC-AUC
        if len(np.unique(numeric_labels)) == 2:
            roc_auc = roc_auc_score(y_test, y_proba[:, 1])
        else:
            roc_auc = roc_auc_score(y_test_binarized, y_proba, multi_class='ovr')

        accuracies.append(acc)
        roc_aucs.append(roc_auc)

        print(f"  Fold {fold}: Accuracy = {acc:.4f}, ROC-AUC = {roc_auc:.4f}")

    avg_acc = np.mean(accuracies)
    avg_roc = np.mean(roc_aucs)
    results[name] = (avg_acc, avg_roc)

    print(f"Average Accuracy: {avg_acc:.4f}, Average ROC-AUC: {avg_roc:.4f}\n")

# Summary Table
print("\nSummary of Results:")
print(f"{'Feature Set':<30} {'Accuracy':<10} {'ROC-AUC':<10}")
for key, (acc, roc) in results.items():
    print(f"{key:<30} {acc:<10.4f} {roc:<10.4f}")

# Best Feature Set
best_feature_set = max(results.items(), key=lambda x: x[1][0])
best_name, (best_acc, best_roc) = best_feature_set

print("\n=== Best Feature Set ===")
print(f"Feature Set: {best_name}")
print(f"Accuracy: {best_acc:.4f}")
print(f"ROC-AUC: {best_roc:.4f}")


Training and evaluating with feature set: TF-IDF
  Fold 1: Accuracy = 0.9200, ROC-AUC = 0.9822
  Fold 2: Accuracy = 0.9200, ROC-AUC = 0.9921
  Fold 3: Accuracy = 1.0000, ROC-AUC = 1.0000
  Fold 4: Accuracy = 0.9600, ROC-AUC = 0.9932
  Fold 5: Accuracy = 0.9100, ROC-AUC = 0.9923
  Fold 6: Accuracy = 0.9200, ROC-AUC = 0.9906
  Fold 7: Accuracy = 0.9200, ROC-AUC = 0.9963
  Fold 8: Accuracy = 0.9000, ROC-AUC = 0.9915
  Fold 9: Accuracy = 0.9500, ROC-AUC = 0.9949
  Fold 10: Accuracy = 0.9100, ROC-AUC = 0.9875
Average Accuracy: 0.9310, Average ROC-AUC: 0.9921


Training and evaluating with feature set: BoW
  Fold 1: Accuracy = 0.9000, ROC-AUC = 0.9803
  Fold 2: Accuracy = 0.9400, ROC-AUC = 0.9936
  Fold 3: Accuracy = 0.9900, ROC-AUC = 0.9999
  Fold 4: Accuracy = 0.9500, ROC-AUC = 0.9926
  Fold 5: Accuracy = 0.8900, ROC-AUC = 0.9924
  Fold 6: Accuracy = 0.9200, ROC-AUC = 0.9949
  Fold 7: Accuracy = 0.9400, ROC-AUC = 0.9984
  Fold 8: Accuracy = 0.9100, ROC-AUC = 0.9931
  Fold 9: Accuracy = 0.